# 1. `ctypes` a `cffi` nad `libmatrix_vector_multiply.so`
Všude držíme stejné ABI (Application Binary Interface) jako v C:

- `float *` <-> NumPy `float32` C-contiguous buffer,
- `size_t` <-> `ctypes.c_size_t` (v Pythonu obyčejné nezáporné `int`),
- návratový typ C funkce je `void`, tedy `restype = None`.

Před voláním vždy kontrolujeme tvar a kompatibilitu dat.


In [ ]:
import ctypes
import numpy as np


def prepare_inputs(matrix, vector):
    matrix_c = np.ascontiguousarray(matrix, dtype=np.float32)
    vector_c = np.ascontiguousarray(vector, dtype=np.float32)

    if matrix_c.ndim != 2:
        raise ValueError("matrix musí být 2D pole")
    if vector_c.ndim != 1:
        raise ValueError("vector musí být 1D pole")
    if matrix_c.shape[1] != vector_c.shape[0]:
        raise ValueError("Počet sloupců matice musí odpovídat délce vektoru")

    result_c = np.zeros(matrix_c.shape[0], dtype=np.float32)
    return matrix_c, vector_c, result_c


matrix, vector, result = prepare_inputs(
    matrix=[[1, 2, 3], [4, 5, 6]],
    vector=[1, 2, 3],
)

lib_matrix_vector_multiply = np.ctypeslib.load_library(
    libname="libmatrix_vector_multiply.so",
    loader_path=".",
)

matrix_vector_multiply = lib_matrix_vector_multiply.matrix_vector_multiply
matrix_vector_multiply.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float32, ndim=2, flags="C_CONTIGUOUS"),
    np.ctypeslib.ndpointer(dtype=np.float32, ndim=1, flags="C_CONTIGUOUS"),
    np.ctypeslib.ndpointer(dtype=np.float32, ndim=1, flags=("C_CONTIGUOUS", "WRITEABLE")),
    ctypes.c_size_t,
    ctypes.c_size_t,
]
matrix_vector_multiply.restype = None

matrix_vector_multiply(matrix, vector, result, matrix.shape[0], matrix.shape[1])
print("ctypeslib:", result)


In [ ]:
matrix @ vector

## 1.1 Totéž přes čisté `ctypes`
`argtypes` definuje přesný seznam C typů argumentů. `restype` definuje návratový typ. Když typy nesedí, chování může být nedefinované i bez okamžité chyby.


In [ ]:
import ctypes

matrix2, vector2, result2 = prepare_inputs(
    matrix=[[1, 2, 3], [4, 5, 6]],
    vector=[1, 2, 3],
)

lib_matrix_vector_multiply = ctypes.CDLL("./libmatrix_vector_multiply.so")
matrix_vector_multiply = lib_matrix_vector_multiply.matrix_vector_multiply
matrix_vector_multiply.argtypes = [
    ctypes.POINTER(ctypes.c_float),
    ctypes.POINTER(ctypes.c_float),
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_size_t,
    ctypes.c_size_t,
]
matrix_vector_multiply.restype = None

matrix_vector_multiply(
    matrix2.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
    vector2.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
    result2.ctypes.data_as(ctypes.POINTER(ctypes.c_float)),
    matrix2.shape[0],
    matrix2.shape[1],
)

print("ctypes:", result2)


## 1.2 Varianta s `cffi`
`cffi` používá stejnou signaturu, jen místo `argtypes` zadáváme C deklaraci přes `ffi.cdef`.

Význam hlavních volání:
- `ffi.cdef(...)`: deklarace C funkcí a typů, které chceme z Pythonu používat,
- `ffi.dlopen(...)`: načtení již zkompilované knihovny,
- `ffi.cast(...)`: explicitní převod adresy bufferu na C pointer odpovídající signatuře.


In [ ]:
%%writefile matrix_vector_multiply_cffi.h
void matrix_vector_multiply(const float *matrix, const float *vector, float *result, size_t rows, size_t cols);


In [ ]:
import numpy as np
from cffi import FFI

ffi = FFI()
with open("matrix_vector_multiply_cffi.h", "r", encoding="utf-8") as f:
    ffi.cdef(f.read())

lib_matrix_vector_multiply = ffi.dlopen("./libmatrix_vector_multiply.so")

matrix3, vector3, result3 = prepare_inputs(
    matrix=[[1, 2, 3], [4, 5, 6]],
    vector=[1, 2, 3],
)

lib_matrix_vector_multiply.matrix_vector_multiply(
    ffi.cast("const float *", matrix3.ctypes.data),
    ffi.cast("const float *", vector3.ctypes.data),
    ffi.cast("float *", result3.ctypes.data),
    matrix3.shape[0],
    matrix3.shape[1],
)

print("cffi:", result3)


In [ ]:
matrix3 @ vector3